In [1]:
# import packages 

import numpy as np
import os
import datetime
import time
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


# folder path and name
project_path = os.getcwd()
data_folder = os.path.join(project_path,"data_expanded")
pred_folder = os.path.join(data_folder,'data_forecast')
pv_data_path = os.path.join(project_path,'pv_data','pv_output_valid.pkl')

image_name_format = '%Y%m%d%H%M%S'

# Operating parameter
stack_height = 15 # 15 minute
forecast_horizon = 15 # 15 minutes ahead forecast
sampling_interval_all = [2]
output_img_shape = [64, 64, 3]

start_date = datetime.datetime(2017,3,9)
end_date = datetime.datetime(2017,12,31)

# Setting up test set
sunny_day = [(2017,3,14),(2017,5,20),(2017,6,4),(2017,7,6),(2017,8,19),(2017,10,7),(2017,11,1),(2017,12,26)]
cloudy_day = [(2017,3,15),(2017,5,24),(2017,7,5),(2017,9,6),(2017,9,22),(2017,11,4),(2017,12,29)]

sunny_datetime = [datetime.datetime(day[0],day[1],day[2]) for day in sunny_day]
cloudy_datetime = [datetime.datetime(day[0],day[1],day[2]) for day in cloudy_day]
test_dates = sunny_datetime + cloudy_datetime

In [2]:
test_dates

[datetime.datetime(2017, 3, 14, 0, 0),
 datetime.datetime(2017, 5, 20, 0, 0),
 datetime.datetime(2017, 6, 4, 0, 0),
 datetime.datetime(2017, 7, 6, 0, 0),
 datetime.datetime(2017, 8, 19, 0, 0),
 datetime.datetime(2017, 10, 7, 0, 0),
 datetime.datetime(2017, 11, 1, 0, 0),
 datetime.datetime(2017, 12, 26, 0, 0),
 datetime.datetime(2017, 3, 15, 0, 0),
 datetime.datetime(2017, 5, 24, 0, 0),
 datetime.datetime(2017, 7, 5, 0, 0),
 datetime.datetime(2017, 9, 6, 0, 0),
 datetime.datetime(2017, 9, 22, 0, 0),
 datetime.datetime(2017, 11, 4, 0, 0),
 datetime.datetime(2017, 12, 29, 0, 0)]

In [3]:
def find_idx_with_dates(all_times,test_dates):
    idx=[]
    for test_day in test_dates:
        test_day_end = test_day + datetime.timedelta(days = 1)
        idx+=np.nonzero((all_times>test_day)*(all_times<test_day_end))[0].tolist()
    return idx

# This two function does the same thing. Just that one is for np, the other for pd.

def find_time_within_nparray(time_array,time_point):
    probable_idx = np.searchsorted(time_array,time_point)
    
    # If the time point is after all the time in pv_data
    if probable_idx == len(time_array):
        return None   
    
    # See if the time point is actually a match 
    if time_array[probable_idx]== time_point: 
        return probable_idx
        
    else:
        return None

def find_time_within_pdseries(time_array,time_point):
    probable_idx = np.searchsorted(time_array,time_point)
    
    # If the time point is after all the time in pv_data
    if probable_idx == len(time_array):
        return None   
    
    # See if the time point is actually a match 
    if time_array[probable_idx] == time_point: 
        return probable_idx
        
    else:
        return None

In [4]:
def store_trainval_test(all_times,image_log,pv_log,pv_pred,pred_folder):
    
    ## Splitting into Trainval and Test set 
    idx_test = find_idx_with_dates(all_times,test_dates)
    image_log_test = image_log[idx_test]
    pv_log_test = pv_log[idx_test]
    pv_pred_test = pv_pred[idx_test]
    times_test = all_times[idx_test]

    # the rest become the trainval set
    mask_trainval = np.ones_like(pv_pred,dtype = bool)
    mask_trainval[idx_test] = 0
    image_log_trainval = image_log[mask_trainval]
    pv_log_trainval = pv_log[mask_trainval]
    pv_pred_trainval = pv_pred[mask_trainval]
    times_trainval = all_times[mask_trainval]
    
    print("times_trainval.shape",times_trainval.shape)
    print("image_log_trainval.shape",image_log_trainval.shape)
    print("pv_log_trainval.shape",pv_log_trainval.shape)
    print("pv_pred_trainval.shape",pv_pred_trainval.shape)
    
    print("times_test.shape",times_test.shape)
    print("image_log_test.shape",image_log_test.shape)
    print("pv_log_test.shape",pv_log_test.shape)
    print("pv_pred_test.shape",pv_pred_test.shape)
    
    #storing information
    # storing the training set
    np.save(os.path.join(pred_folder,'image_log_trainval.npy'), image_log_trainval)
    np.save(os.path.join(pred_folder,'pv_log_trainval.npy'), pv_log_trainval)
    np.save(os.path.join(pred_folder,'pv_pred_trainval.npy'),pv_pred_trainval)
    np.save(os.path.join(pred_folder,'times_trainval.npy'),times_trainval)

    # storing the testing set
    np.save(os.path.join(pred_folder,'image_log_test.npy'), image_log_test)
    np.save(os.path.join(pred_folder,'pv_log_test.npy'), pv_log_test)
    np.save(os.path.join(pred_folder,'pv_pred_test.npy'),pv_pred_test)
    np.save(os.path.join(pred_folder,'times_test.npy'),times_test)

In [5]:
# Load in  high frequency data
# The image here are ones that have corresponding PV value
all_times = np.load(os.path.join(data_folder,'all_times_highfreq.npy'), allow_pickle=True)
all_images = np.load(os.path.join(data_folder,'all_images_highfreq.npy'), allow_pickle=True)
pv_data = np.load(pv_data_path, allow_pickle=True)

# only pick out the relevant time period
relevant_mask = (all_times>start_date)&(all_times<end_date)
all_times = all_times[relevant_mask]
all_images = all_images[relevant_mask]
pv_data = pv_data[start_date:end_date]

n_images = all_times.shape[0]

In [6]:
# Create forecast training data file
for sampling_interval in sampling_interval_all:
    # Initialize variables to save pv values
    image_log = np.zeros([n_images,stack_height+1]+output_img_shape,dtype = 'uint8')
    pv_log = np.zeros((n_images,stack_height+1))
    pv_pred = np.zeros(n_images)
    validity_mask = np.ones(n_images,dtype = bool)
    tic = time.process_time()
    last_valid_index = 0

    sampling_interval_td = datetime.timedelta(minutes = sampling_interval) - datetime.timedelta(seconds=1)
    for i in range(0,n_images):

        # See if the specified sampling frequency is met 
        if all_times[i] - all_times[last_valid_index] > sampling_interval_td:

            # Collecting groud truth for predicted value
            pred_time = all_times[i]+datetime.timedelta(minutes=forecast_horizon)
            
            pv_pred_idx = find_time_within_nparray(pv_data.index,pred_time)
            if pv_pred_idx is None:# if prediction ground truth not found
                validity_mask[i] = False
                print(all_times[i],'has no PV pred')
            else: 
                pv_pred[i] = pv_data.iloc[pv_pred_idx] 

            # Collecting image log and PV log
            for j in range(stack_height+1):
                log_time = all_times[i] - datetime.timedelta(minutes = j)
                # Collecting a stack of image
                log_time_idx = find_time_within_nparray(all_times,log_time)
                if log_time_idx is not None:
                    image_log[i,j] = all_images[log_time_idx]
                else:
                    validity_mask[i] = False
                    print(all_times[i],'has no image log')
                    break

                # Collecting a stack of PV value
                pv_log_idx = find_time_within_nparray(pv_data.index,log_time)
                # Check if PV value present
                if pv_log_idx is None:
                    validity_mask[i] = False
                    print(all_times[i],'has no PV log')
                    break
                else: 
                    pv_log[i,j] = pv_data.iloc[pv_log_idx]    

        else: # if this is in between the sampling points, discard
            validity_mask[i] = False
        
        if validity_mask[i]:
            last_valid_index = i
        
        # Prompt progress of current work
        if i%100 == 0:
            print('processed {0}/{1} images'.format(i,len(all_times)))
            if i%1000 == 0 and i>0:
                print('For sampling frequency: ',sampling_interval,' minutes')
                print('Expected finishing time:', datetime.datetime.now()+
                           datetime.timedelta(seconds = (time.process_time() - tic)*(len(all_times)/i-1)))
    
    # Only pick out the valid time points
    all_times = all_times[validity_mask]
    image_log = image_log[validity_mask]
    pv_log = pv_log[validity_mask]
    pv_pred = pv_pred[validity_mask]
    
    # Store information
    pred_folder_child = os.path.join(pred_folder,'frequency_'+str(sampling_interval))
    store_trainval_test(all_times,image_log,pv_log,pv_pred,pred_folder_child)

processed 0/107992 images
2017-03-09 06:49:00 has no image log
2017-03-09 06:50:00 has no image log
2017-03-09 06:51:00 has no image log
2017-03-09 06:52:00 has no image log
2017-03-09 06:53:00 has no image log
2017-03-09 06:54:00 has no image log
2017-03-09 06:55:00 has no image log
2017-03-09 06:56:00 has no image log
2017-03-09 06:57:00 has no image log
2017-03-09 06:58:00 has no image log
2017-03-09 06:59:00 has no image log
2017-03-09 07:00:00 has no image log
2017-03-09 07:01:00 has no image log
processed 100/107992 images
2017-03-09 08:42:30 has no image log
2017-03-09 08:43:30 has no image log
2017-03-09 08:44:30 has no image log
2017-03-09 08:45:30 has no image log
2017-03-09 08:46:30 has no image log
2017-03-09 08:47:30 has no image log
2017-03-09 08:48:30 has no image log
2017-03-09 08:49:30 has no image log
2017-03-09 08:50:30 has no image log
2017-03-09 08:51:30 has no image log
2017-03-09 08:52:30 has no image log
2017-03-09 08:53:30 has no image log
2017-03-09 08:54:30 h